In [ ]:
import sys
import xlrd
import os
import numpy as np
import matplotlib as mpl
mpl.use('Agg')  # standard rendering tool for matplotlib above
import matplotlib.pyplot as plt
from scipy import optimize
import scipy.stats as st
from matplotlib import rcParams


# retrieve xlsx file, user inputs name of file
location = raw_input("What is the name of your excel file?")
file_location = str(location) + ".xlsx"
workbook = xlrd.open_workbook(file_location)
sheet = workbook.sheet_by_index(0)

# Determine peak threshold
user_threshold = float(raw_input("Enter % for threshold (1-100)"))
# create blank lists to store excel columns
time = []
data = []

# retrieve time (col A) and data (col B)
for row in range(sheet.nrows):
    time.append(sheet.cell_value(row, 0))
    data.append(sheet.cell_value(row, 1))

# convert the lists to NumPy arrays, which are much faster, and can be
# passed as parameters to NumPy/SciPy matrix functions

actual_time = time
actual_data = data


# call z to cumulative proportion and vice versa
# input st.norm.ppf(.95)
# return 1.6448536269514722
# input st.norm.cdf(1.64)
# return 0.94949741652589625
# normal function: ((1 / (np.sqrt(2 * np.pi * 1 ** 2))) * (np.e ** ((-1) * (((0 - 0) ** 2) / (2 * 1 ** 2)))))
# sample dataset = home1
# convert to total a6mt metabolite (data are in ng/h; desire ng

def normal(x, mu, sigma):
    normal_ht = (1 / (np.sqrt(2 * np.pi * sigma ** 2))) * (np.e ** ((-1) * (((x - mu) ** 2) / (2 * sigma ** 2))))
    return normal_ht


def std_normal(x):
    std_normal_ht = (1 / (np.sqrt(2 * np.pi * 1 ** 2))) * (np.e ** ((-1) * (((x - 0) ** 2) / (2 * 1 ** 2))))
    return std_normal_ht


total_ng_list = []

# function serving to convert ng per interval to total cumulative ng
def total_ng(time, data):
    total_sum = 0
    total_ng_list.append(0)
    for i in range(len(time)-1):
        mp_sum = (time[i+1] - time[i])*data[i + 1]
        total_sum += mp_sum
        total_ng_list.append(total_sum)
    return total_sum


total = total_ng(time, data)

# convert cumulative ng produced to proportion of total
prop_list = []


def find_totals(x):
    for totals in x:
        props = totals / total
        prop_list.append(props)
    return prop_list


actual_cumulative_proportions = find_totals(total_ng_list)


# input cumulative proportion list, convert to z score list
z_list = []


def get_z(x):
    for prop in x:
        z = st.norm.ppf(prop)
        z_list.append(z)
    return z_list


actual_z_scores = get_z(prop_list)


# find y values for each z score in actual z scores list
actual_z_ht = []


for z in actual_z_scores:
    ht = ((1 / (np.sqrt(2 * np.pi * 1 ** 2))) * (np.e ** ((-1) * (((z - 0) ** 2) / (2 * 1 ** 2)))))
    actual_z_ht.append(ht)


# determine values: time (hr) and z, occurring directly prior and directly after, the 50% ng total
# solve for linear (actual point to actual point) 50% total time aka acrophase
linear_acrophase = 0
pre_mid_z = 0
pre_mid_time = 0
post_mid_z = 0
post_mid_time = 0
prop_step = 0

while prop_step < len(total_ng_list):
    if total_ng_list[prop_step] < total/2:
        if total_ng_list[prop_step+1] > total/2:
            linear_acrophase = (0.5*total-(total_ng_list[prop_step+1]-((total_ng_list[prop_step+1]-total_ng_list[prop_step])/
                                                                  (time[prop_step+1]-time[prop_step]))*time[prop_step+1]))/((total_ng_list[prop_step+1]-total_ng_list[prop_step])/(time[prop_step+1]-time[prop_step]))
            pre_mid_time = time[prop_step]
            post_mid_time = time[prop_step+1]
            pre_mid_z = z_list[prop_step]
            post_mid_z = z_list[prop_step+1]
            break
    prop_step += 1

# ignoring linearly determined acrophase, algebraically solve for sigma, then mu, of a normal distribution using only
# time, z scores of datapoints directly preceding and proceeding the 50% ng total
normal_sigma = (post_mid_time - pre_mid_time) / (post_mid_z - pre_mid_z)
normal_acrophase = post_mid_time - post_mid_z * normal_sigma
normal_acrophase_ht = normal(normal_acrophase, normal_acrophase, normal_sigma)
fitted_acrophase_ht = normal_acrophase_ht*total
# saving this normal distribution formula for future use; pay no attention!
#(((1 / (np.sqrt(2 * np.pi * 1 ** 2))) * (np.e ** ((-1) * (((0 - 0) ** 2) / (2 * 1 ** 2)))))/((1 / (np.sqrt(2 * np.pi * 1 ** 2))) * (np.e ** ((-1) * (((pre_mid_z - 0) ** 2) / (2 * 1 ** 2))))))

# given computed mu and sigma of fitted normal distribution, fit z scores onto actual data
fit_z = []
for hr in actual_time:
    fitted_z = (hr - normal_acrophase)/normal_sigma
    fit_z.append(fitted_z)

# given computed mu and sigma of fitted normal distribution, fit y values onto actual data
fit_ht = []
for z in actual_time:
    ht = normal(z, normal_acrophase, normal_sigma)
    fit_ht.append(ht)

# given computed mu and sigma of fitted normal distribution, fit cumulative proportions onto actual data
fit_cdf = []
for z in fit_z:
    cdf = st.norm.cdf(z)
    fit_cdf.append(cdf)

# given computed mu and sigma of fitted normal distribution, fit real y values (total ng * z ht) to actual data
fit_ng = []
for ht in fit_ht:
    ngs = total*ht
    fit_ng.append(ngs)

# determine threshold for onset and offset
fitted_midpoint = fitted_acrophase_ht*(user_threshold/100)

# list below is for figure display purposes only
fitted_midpoint_list = []
for items in time:
    fitted_midpoint_list.append(fitted_midpoint)

# prepare actual time and data to estimate ng/h values in order to compare to actual values, to generate SSresiduals
time_fit = time
data_fit = data
data_fit_ng_h = []

cum_step = 1
data_fit_ng_h.append(0)
while cum_step < len(data):
    ngh = total*(fit_cdf[cum_step]-fit_cdf[cum_step-1])/(time[cum_step]-time[cum_step-1])
    data_fit_ng_h.append(ngh)
    cum_step += 1

data_fit_ng_h = np.array(data_fit_ng_h)

# mesor-data intersection points
time = np.array(time)
data = np.array(data)

# below returns data index points prior to mesor crossing, writes these index values to y_int
idx = np.argwhere(np.diff(np.sign(fitted_midpoint - data))).flatten()
x_int = time[idx]
y_int = []
for coords in x_int:
    y_int.append(fitted_midpoint)

crossing_points = []

# finds time values where mesor intersects with data (assuming straight line from point to point) \
# by generating straight line y = mx+b from two known points at each interval, and inverse to solve for y w/known time
# writes each of these intersection timepoints to crossing_points list (y value is always mesor, this list is x "time")
for intersect in idx:
    crossing_points.append((fitted_midpoint-((data[intersect])-((data[intersect+1] - data[intersect])/(time[intersect+1] -
                                                                                                 time[intersect])) *
                                       time[intersect]))/((data[intersect+1] - data[intersect])/(time[intersect+1] -
                                                                                                 time[intersect])))

crossing_points = np.array(crossing_points)
crossing_points_unstring = ", ".join("{0:.3f}".format(num) for num in crossing_points)

def pearson(x, y):
    # computes pearson correlation coefficient (r) and coefficient of determination (r^2) between arrays x and y.
    # also computes means (xmean, ymean), sums of squares (SSx, SSy) standard deviations (SDy, SDy), /
    # covariance (COVxy) and degrees of freedom (rdf).

    xlen = len(x)
    ylen = len(y)

    xsum = float(0)
    ysum = float(0)

    for values in x:
        xsum += float(values)
    for values in y:
        ysum += float(values)

    xmean = float(xsum/xlen)
    ymean = float(ysum/ylen)
    ss_x = float(0)
    ss_y = float(0)

    for values in x:
        ss_x += float((values-xmean)**2)
    for values in y:
        ss_y += float((values-ymean)**2)

    sd_x = float(ss_x/xlen)**.5
    sd_y = float(ss_y/ylen)**.5

    cov_xy = 0
    rdf = len(x) - 2

    for a, b in zip(x, y):
        cov_xy += float((a-xmean)*(b-ymean))
    # convert this to str containing dfs?
    return float((cov_xy/xlen)/(sd_x*sd_y))


correlation = pearson(data[1:], data_fit_ng_h[1:])
r_squared = correlation**2

ss = 0
ss_step = 1
while ss_step < len(data):
    ss += (data[ss_step]-data_fit_ng_h[ss_step])**2
    ss_step += 1

# tuple pairing time and data values (line 168 of ski_slope_least_squares_1_oct
time_data_tuple = zip(time, data)

# combine time,data coords with crossing_points, lsq_mesor list, then arrange by time
crossing_coords = sorted((zip(crossing_points, fitted_midpoint_list)) + time_data_tuple)
sorted_coords = zip(*crossing_coords)
# all_time is every original timepoint plus mesor intersection timepoints
all_time = sorted_coords[0]
# all_data is every original datapoint plus mesor value when mesor intersects data
all_data = sorted_coords[1]

# find position in sorted coords where crossing points appear (start and end of each auc computation)

index_coords = []

for pts in crossing_points:
    for items in all_time:
        if items == pts:
            index_coords.append(all_time.index(items))

# introduce lists that will be written as a function of whether curve is going up during mesor crossing (onset) \
# or down during crossing (offset) These are onset_coords and offset_coords, respectively.
# onset and offset index_coords display the index number of these locations relative to position in all_time
onset_coords = []
onset_index_coords = []
offset_coords = []
offset_index_coords = []
peak_data_list = []
peak_time_list = []

for pts in index_coords[0:(len(index_coords)-1)]:
    if all_data[pts+1] > all_data[pts]:
        onset_coords.append(pts)
        onset_index_coords.append(index_coords.index(pts))

for pts in index_coords[1:len(index_coords)]:
    if all_data[pts+1] < all_data[pts]:
        offset_coords.append(pts)
        offset_index_coords.append(index_coords.index(pts))


# filters all_time, and all_data arrays into arrays of lists, each sub-list beginning at onset, ending at offset
# all other coordinates ignored
# time list written to peak_time_list, data written to peak_data_list

step = 0
while step < (len(index_coords)):
    if len(index_coords) <= 1:
        print "only 1 mesor crossing; cannot compute peak duration"
        step = len(index_coords)
    elif index_coords[0] == onset_coords[0] and len(index_coords) % 2 == 0 and len(index_coords) >= 2:
        peak_data_list.append(all_data[index_coords[step]:index_coords[step + 1]+1])
        peak_time_list.append(all_time[index_coords[step]:index_coords[step + 1]+1])
        step += 2
    elif index_coords[0] == onset_coords[0] and len(index_coords) % 2 != 0 and len(index_coords) >= 3:
        index_coords.pop()
        peak_data_list.append(all_data[index_coords[step]:index_coords[step + 1] + 1])
        peak_time_list.append(all_time[index_coords[step]:index_coords[step + 1] + 1])
        step += 2
    elif index_coords[0] != onset_coords[0] and len(index_coords) % 2 == 0 and len(index_coords) >= 3:
        index_coords = index_coords[1:-1]
    elif index_coords[0] != onset_coords[0] and len(index_coords) % 2 != 0 and len(index_coords) >= 3:
        index_coords = index_coords[1:]
    elif index_coords[0] != onset_coords[0] and len(index_coords) % 2 == 0 and len(index_coords) <= 2:
        print "not enough coordinates to determine"
    else:
        print "on-off coordinates not found"
        step = len(index_coords)


# midpoint auc calculation
def midpoint_peak_auc(time, data):
    total_sum = 0
    for i in range(len(time)-1):
        mp_sum = (time[i+1] - time[i])*((data[i] + data[i + 1])/2)
        total_sum += mp_sum
    return total_sum


# write all peak_mp_auc calculations to a list
mp_cycle = 0
peak_mp_auc_list = []

while mp_cycle < len(peak_time_list):
    peak_mp_auc_list.append(midpoint_peak_auc(peak_time_list[mp_cycle], peak_data_list[mp_cycle]))
    mp_cycle += 1

# determine actual data determined onset, offset, and peak duration; transform threshold cross times to printable format
peak_duration = crossing_points[1] - crossing_points[0]
peak_duration_unstring = float(peak_duration)
peak_mp_auc_list_unstring = ", ".join("{0:.3f}".format(num) for num in peak_mp_auc_list)
onset = crossing_points[0]
offset = crossing_points[1]


# Find threshold crossings of normal distribution and corresponding z score
fitted_offset = np.sqrt((2 * normal_sigma**2)*(-1*np.log((fitted_midpoint*np.sqrt(2*np.pi*normal_sigma**2))/total))) + normal_acrophase
fitted_onset = (normal_acrophase - fitted_offset) + normal_acrophase
fitted_duration = fitted_offset - fitted_onset
threshold_z = (fitted_offset - normal_acrophase)/normal_sigma

# setup the figure plots to be generated
rcParams["figure.figsize"] = (10, 14)
rcParams["legend.fontsize"] = 16
rcParams["axes.labelsize"] = 16

%matplotlib notebook

# create a blank figure
fig = plt.figure()

# add a plot (2x1 grid in the 1st position)
axes = fig.add_subplot(211)

# generate smooth fitted curves by upping the resolution to 100
time_fit = np.linspace(time.min(), time.max(), 100)
data_fit = total*(normal(time_fit, normal_acrophase, normal_sigma))
acro_x_y = (normal_acrophase, fitted_acrophase_ht)
# plot the data ("ro" = red circles) and the fit ("r-" = red line)
axes.plot(time, data, "k-")
axes.plot(time, data, "ko", label="Actual Data")
axes.plot(time_fit, data_fit, "r-", label="ng Fit")
axes.plot(time, data_fit_ng_h, "g-", label="ng/h Fit")
axes.plot(normal_acrophase, fitted_acrophase_ht, "ro")
plt.hlines(fitted_midpoint_list, time[0], time[len(time)-1], "y", label="Peak Threshold")
axes.plot(crossing_points, y_int, "yo")
axes.plot(fitted_onset, fitted_midpoint, "yo")
axes.plot(fitted_offset, fitted_midpoint, "yo")

# add a legend
axes.set_title("Z Distribution Fit")
axes.set_xlabel("Hour")
axes.set_ylabel("aMT6")
axes.legend()

# add a text box below the graph
axes2 = fig.add_subplot(212)
axes2.axis("off")

# create function with properly formatted output underneath figure
def append_text(x_pos, y_pos, ss, num_residuals, r, r2, acro_x, acro_y, mesor, peak_duration_unstring, cross_time, auc, color, mu, sigma, total, onset, offset, threshold, fit_on, fit_off, fit_duration, fit_z):
    """Appends the residual data below the chart
    x_pos         -- x-position
    y_pos         -- y-position
    num_residuals -- number of residuals
    r             -- sum of residuals
    r2            -- sum of squared residuals
    """
    strfmt = "h = {1:.4f}, b = {2:.4f}, v = {3:.4f}, p = {4:.4f}"
    axes2.text(x_pos, y_pos,
            # surrounding text in $-symbols will format it as math equations
            "$r({1:d}) = {2:,.3f}$,  $r^2({1:d}) = {3:,.3f}$, Sum of Squares (SS) = {0:,.3f}\n\nmu = {10:.3f} h\nsigma = {11:.3f} h\n\nTotal aMT6 = {12:.3f} ng\n\nPeak Coordinates = ({4:,.3f}, {5:,.3f})\n\nAcrophase = {4:,.3f} h, Max Value = {5:,.3f} ng\n\n{15:,.1f}% Threshold = {6:,.3f} ng\n\nOnset = {13:,.3f} h\nOffset = {14:,.3f} h \nPeak Duration = {7:,.3f} h\n\nFitted Onset = {16:.3f} h\nFitted Offset = {17:.3f} h\nFitted Duration = {18:.3f} h\nFitted Z Threshold = +/-{19:.3f}".format
               (ss, num_residuals - 3, r, r2, acro_x, acro_y, mesor, peak_duration_unstring, cross_time, auc, mu, sigma, total, onset, offset, threshold, fit_on, fit_off, fit_duration, fit_z),
            color=color,
            fontsize=16,
            horizontalalignment="left",
            verticalalignment="top",
            wrap=True,
            transform=axes2.transAxes)


# insert residual info along the left side of text box
append_text(.02, .96, ss, len(data), correlation, r_squared, normal_acrophase, fitted_acrophase_ht, fitted_midpoint,
           peak_duration_unstring, crossing_points_unstring, peak_mp_auc_list_unstring, "k", normal_acrophase, normal_sigma, total, onset, offset, user_threshold, fitted_onset, fitted_offset, fitted_duration, threshold_z),


# save the figure as an image file in the current directory
# fig.savefig(os.path.join(os.getcwd(), str(location)+"_z_dist_output.png"))
plt.show()


print "Original (time, aMT6 ng/h) coordinates:"
print zip(time, data)
print "Total ng secreted = %.3f ng" % total
print "pre mid time: %s, pre mid z: %.3f, post mid time: %s, post mid z: %.3f" % (pre_mid_time, pre_mid_z, post_mid_time, post_mid_z)
print "Actual Z-scores: %s" % actual_z_scores
print "Actual cumulative proportions: %s" % actual_cumulative_proportions
print "Actual Standard Normal Ht: %s" % actual_z_ht
print "Linear Acrophase: %.3f" % linear_acrophase
print "Normal Distribution Mu = %.3f, sigma = %.3f" % (normal_acrophase, normal_sigma)
print "Normal Acrophase ht: %.3f" % normal_acrophase_ht
print "Fitted Acrophase ht: %.3f" % fitted_acrophase_ht
print "Fitted Z: %s" % fit_z
print "Fitted cumulative proportions: %s" % fit_cdf
print "Fitted ht: %s" % fit_ht
print "Fitted ng = %s" % fit_ng
print "SS = %.3f" % ss
print "Fitted ng/h = %s" % data_fit_ng_h
print fitted_onset
print fitted_offset
print "Threshold z = +/-%.3f" % threshold_z

